# MODEL A: 75x75 model

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
datagen = ImageDataGenerator(rescale = .1/255, rotation_range = 40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

train_generator = datagen.flow_from_directory(
    'train',
    target_size=(75, 75),
    class_mode='categorical',
    batch_size=400  # Set batch size to match the total number of images
)

Found 9112 images belonging to 5 classes.


In [3]:

model = tf.keras.applications.InceptionV3(input_shape=(75, 75, 3), weights='imagenet', include_top=False)

# Freeze the weights of all the layers except for the last one
for layer in model.layers[:-1]:
    layer.trainable = False

# Add a new output layer
x = tf.keras.layers.Flatten()(model.output)
x = tf.keras.layers.Dense(1024,"relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(5, "softmax")(x)

# Compile the model
model = tf.keras.Model(model.input, x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

2023-10-20 18:12:04.201269: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-10-20 18:12:04.201298: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-10-20 18:12:04.201308: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-10-20 18:12:04.201538: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-20 18:12:04.201762: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 75, 75, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 37, 37, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 37, 37, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 37, 37, 32)           0         ['batch_normalization[0][0

In [ ]:
# Train the model
# TODO: add checkpointing
model.fit_generator(train_generator, epochs=30)

In [ ]:
model.save("models/modelA_75.h5")